## 2. train

In [1]:
import argparse
import logging
import os
import os.path as osp
import sys

import numpy as np
import torch
import torch.nn as nn
import torchvision as tv
from torch import optim
from torch.cuda import amp
from torch.nn.modules import activation
from torch.nn.modules.activation import Threshold
from tqdm import tqdm

from eval import eval_net

from torch.utils.data import DataLoader, random_split
from torch.utils.data.distributed import DistributedSampler
import utils
import models
from utils.DriveDataset import DriveDataset

logger = logging.getLogger(__name__)

dir_img = osp.join(".", "dataset", "train")
dir_mask = osp.join(".", "dataset", "train_GT")

# dir_img = osp.join(".", "dataset0", "train")
# dir_mask = osp.join(".", "dataset0", "train_GT")

def is_parallel(model):
    return type(model) in (nn.parallel.DataParallel,
                           nn.parallel.DistributedDataParallel)

def get_args():
    parser = argparse.ArgumentParser(
        description='Train the UNet on images and target masks',
        formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    parser.add_argument('-e',
                        '--epochs',
                        metavar='E',
                        type=int,
                        default=1,
                        help='Number of epochs',
                        dest='epochs')
    parser.add_argument('-b',
                        '--batch_size',
                        metavar='B',
                        type=int,
                        nargs='?',
                        default=1,
                        help='Batch size',
                        dest='batchsize')
    parser.add_argument('-l',
                        '--learning_rate',
                        metavar='LR',
                        type=float,
                        nargs='?',
                        default=0.0001,
                        help='Learning rate',
                        dest='lr')
    parser.add_argument('-f',
                        '--load',
                        dest='load',
                        default=False,
                        action='store_true',
                        help='Load model from a .pth file')
    parser.add_argument('-s',
                        '--scale',
                        dest='scale',
                        type=float,
                        default=0.5,
                        help='Downscaling factor of the images')
    parser.add_argument('-v',
                        '--validation',
                        dest='val',
                        type=float,
                        default=0.1,
                        help='Percent of the data \
                              that is used as validation (0-100)')
    parser.add_argument('-d',
                        '--device',
                        default='cpu',
                        help='cuda device, i.e. 0 or 0,1,2,3 or cpu')
    parser.add_argument('--local_rank',
                        type=int,
                        default=-1,
                        help='DDP parameter, do not modify')
    parser.add_argument('--model_type',
                        type=str,
                        default='utrans',
                        help="Model which choosed.")
    parser.add_argument('--split_seed', type=int, default=None, help='')
    return parser.parse_args(args=[])

def select_device(device='', batch_size=None):
    # device = 'cpu' or '0' or '0,1,2,3'
    s = f'UNetHX torch {torch.__version__} '
    cpu = device.lower() == 'cpu'
    if cpu:
        os.environ[
            'CUDA_VISIBLE_DEVICES'] = '-1'  # force torch.cuda.is_available() = False
    elif device:  # non-cpu device requested
        os.environ['CUDA_VISIBLE_DEVICES'] = device  # set environment variable
        assert torch.cuda.is_available(
        ), f'CUDA unavailable, invalid device {device} requested'  # check availability

    cuda = not cpu and torch.cuda.is_available()
    if cuda:
        n = torch.cuda.device_count()
        if n > 1 and batch_size:  # check that batch_size is compatible with device_count
            assert batch_size % n == 0, f'batch-size {batch_size} not multiple of GPU count {n}'
        space = ' ' * len(s)
        for i, d in enumerate(device.split(',') if device else range(n)):
            p = torch.cuda.get_device_properties(i)
            s += f"{'' if i == 0 else space}CUDA:{d} ({p.name}, {p.total_memory / 1024 ** 2}MB)\n"  # bytes to MB
    else:
        s += 'CPU\n'

    logger.info(s)  # skip a line
    return torch.device('cuda:0' if cuda else 'cpu')

ROOT_FOLDER = "./DRIVE/"
task = "training" # "test"

def train_net(model,
              device,
              epochs=5,
              batch_size=1,
              lr=0.001,
              val_percent=0.1,
              save_all_cp=True,
              dir_checkpoint='runs',
              split_seed=None):
    transform_valid = tv.transforms.Compose([tv.transforms.ToTensor(),
                                             tv.transforms.RandomCrop((400, 400))
                                             ])
    # dataset = BasicDataset(dir_img, dir_mask, transform=transform_valid)
    dataset = DriveDataset(task, ROOT_FOLDER, transform=transform_valid)

    n_val = int(len(dataset) *
                val_percent) if val_percent < 1 else int(val_percent)
    n_train = len(dataset) - n_val
    if split_seed:
        train, val = random_split(
            dataset, [n_train, n_val],
            generator=torch.Generator().manual_seed(split_seed))
    else:
        train, val = random_split(dataset, [n_train, n_val])
    if type(model) == nn.parallel.DistributedDataParallel:
        train_loader = DataLoader(train,
                                  batch_size=batch_size,
                                  shuffle=False,
                                  num_workers=0,
                                  pin_memory=True,
                                  sampler=DistributedSampler(train))
        val_loader = DataLoader(val,
                                batch_size=batch_size,
                                shuffle=False,
                                num_workers=0,
                                pin_memory=True,
                                drop_last=True,
                                sampler=DistributedSampler(val))
    else:
        train_loader = DataLoader(train,
                                  batch_size=batch_size,
                                  shuffle=True,
                                  num_workers=0,
                                  pin_memory=True)
        val_loader = DataLoader(val,
                                batch_size=batch_size,
                                shuffle=False,
                                num_workers=0,
                                pin_memory=True,
                                drop_last=True)

    logging.info(f'''Starting training:
        Epochs:          {epochs}
        Batch size:      {batch_size}
        Learning rate:   {lr}
        Training size:   {n_train}
        Validation size: {n_val}
        Checkpoints:     {save_all_cp}
        Device:          {device.type}
    ''')

    # loss = nn.BCEWithLogitsLoss()
    # loss.__name__ = 'BCEWithLogitLoss'
    # loss = nn.BCELoss()
    # loss.__name__ = 'BCELoss'
    loss = utils.losses.NoiseRobustDiceLoss(eps=1e-7, activation='sigmoid')
    metrics = [
        utils.metrics.Dice(threshold=0.5, activation='sigmoid'),
        utils.metrics.Fscore(threshold=None, activation='sigmoid')
    ]
    optimizer = torch.optim.Adam([
        dict(params=model.parameters(), lr=lr),
    ])

    train_epoch = utils.train.TrainEpoch(
        model,
        loss=loss,
        metrics=metrics,
        optimizer=optimizer,
        device=device,
        verbose=True,
    )
    valid_epoch = utils.train.ValidEpoch(
        model,
        loss=loss,
        metrics=metrics,
        device=device,
        verbose=True,
    )

    max_score = 0
    os.makedirs(dir_checkpoint, exist_ok=True)
    for i in range(0, epochs):
        print('\nEpoch: {}'.format(i + 1))
        train_logs = train_epoch.run(train_loader)
        valid_logs = valid_epoch.run(val_loader)

        # do something (save model, change lr, etc.)
        if max_score < valid_logs['dice_score']:
            max_score = valid_logs['dice_score']
            torch.save(model, osp.join(dir_checkpoint, 'best_model.pt'))
            torch.save(model.state_dict(),
                       osp.join(dir_checkpoint, 'best_model_dict.pth'))
            print('Model saved!')

        if save_all_cp:
            torch.save(model.state_dict(),
                       osp.join(dir_checkpoint, f'CP_epoch{i + 1}.pth'))

C:\Anaconda\envs\CMIT\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")


## Start

In [2]:
logging.basicConfig(level=logging.INFO,
                    format='%(levelname)s: %(message)s')
args = get_args()
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = select_device(args.device, batch_size=args.batchsize)
logging.info(f'Using device {device}')

import socket
from datetime import datetime
current_time = datetime.now().strftime('%b%d_%H-%M-%S')
comment = f'MT_{args.model_type}_SS_{args.split_seed}_LR_{args.lr}_BS_{args.batchsize}'
dir_checkpoint = osp.join(
    ".", "checkpoints",
    f"{current_time}_{socket.gethostname()}_" + comment)

nets = {
    # "unet": models.UNet,
    # "inunet": InUNet,
    # "attunet": AttU_Net,
    # "inattunet": InAttU_Net,
    # "att2uneta": Att2U_NetA,
    # "att2unetb": Att2U_NetB,
    # "att2unetc": Att2U_NetC,
    # "ecaunet": ECAU_Net,
    # "gsaunet": GsAUNet,
    # "utnet": U_Transformer,
    #"ddrnet": models.DualResNet,
    "utrans": models.U_Transformer
}

net_type = nets[args.model_type.lower()]
net = net_type(in_channels=3, classes=1)

net.to(device=device)

cuda = device.type != 'cpu'

if cuda and args.local_rank == -1 and torch.cuda.device_count() > 1:
    print(f"DP Use multiple gpus: {args.device}")
    net = nn.DataParallel(net)
# DDP mode
if cuda and args.local_rank != -1 and torch.cuda.device_count() > 1:
    print(f"DDP Use multiple gpus: {args.device}")
    assert torch.cuda.device_count() > args.local_rank
    device = torch.device('cuda', args.local_rank)
    torch.distributed.init_process_group(backend="nccl")
    net = nn.parallel.DistributedDataParallel(net)
net = net.to(device=device)

net = net.module if is_parallel(net) else net
net = net.to(device=device)

if args.load:
    net.load_state_dict(torch.load(args.load, map_location=device))
    logging.info(f'Model loaded from {args.load}')

INFO: UNetHX torch 1.12.0 CPU

INFO: Using device cpu


In [3]:
train_net(model=net,
            epochs=args.epochs,
            batch_size=args.batchsize,
            lr=args.lr,
            device=device,
            val_percent=args.val,
            dir_checkpoint=dir_checkpoint,
            split_seed=args.split_seed,
            save_all_cp=True)

INFO: Starting training:
        Epochs:          1
        Batch size:      1
        Learning rate:   0.0001
        Training size:   18
        Validation size: 2
        Checkpoints:     True
        Device:          cpu
    



Epoch: 1
train:   0%|          | 0/18 [00:00<?, ?it/s][  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 

KeyError: 'dice_score'

Test

In [96]:
model=net
device
epochs=5
batch_size=1
lr=0.001
val_percent=0.1
save_all_cp=True
dir_checkpoint='runs'
split_seed=None

transform_valid = tv.transforms.Compose([tv.transforms.ToTensor(),
                                         tv.transforms.RandomCrop((400, 400))
                                         ])
# dataset = BasicDataset(dir_img, dir_mask, transform=transform_valid)
dataset = DriveDataset(task, ROOT_FOLDER, transform=transform_valid)

n_val = int(len(dataset) *
            val_percent) if val_percent < 1 else int(val_percent)
n_train = len(dataset) - n_val
if split_seed:
    train, val = random_split(
        dataset, [n_train, n_val],
        generator=torch.Generator().manual_seed(split_seed))
else:
    train, val = random_split(dataset, [n_train, n_val])
if type(model) == nn.parallel.DistributedDataParallel:
    train_loader = DataLoader(train,
                              batch_size=batch_size,
                              shuffle=False,
                              num_workers=0,
                              pin_memory=True,
                              sampler=DistributedSampler(train))
    val_loader = DataLoader(val,
                            batch_size=batch_size,
                            shuffle=False,
                            num_workers=0,
                            pin_memory=True,
                            drop_last=True,
                            sampler=DistributedSampler(val))
else:
    train_loader = DataLoader(train,
                              batch_size=batch_size,
                              shuffle=True,
                              num_workers=0,
                              pin_memory=True)
    val_loader = DataLoader(val,
                            batch_size=batch_size,
                            shuffle=False,
                            num_workers=0,
                            pin_memory=True,
                            drop_last=True)

logging.info(f'''Starting training:
    Epochs:          {epochs}
    Batch size:      {batch_size}
    Learning rate:   {lr}
    Training size:   {n_train}
    Validation size: {n_val}
    Checkpoints:     {save_all_cp}
    Device:          {device.type}
''')

# loss = nn.BCEWithLogitsLoss()
# loss.__name__ = 'BCEWithLogitLoss'
# loss = nn.BCELoss()
# loss.__name__ = 'BCELoss'
loss = utils.losses.NoiseRobustDiceLoss(eps=1e-7, activation='sigmoid')
metrics = [
    utils.metrics.Dice(threshold=0.5, activation='sigmoid'),
    utils.metrics.Fscore(threshold=None, activation='sigmoid')
]
optimizer = torch.optim.Adam([
    dict(params=model.parameters(), lr=lr),
])


INFO: Starting training:
    Epochs:          5
    Batch size:      1
    Learning rate:   0.001
    Training size:   18
    Validation size: 2
    Checkpoints:     True
    Device:          cpu



### train_epoch

In [97]:
loss = utils.losses.NoiseRobustDiceLoss(eps=1e-7, activation='sigmoid')
metrics = [
    utils.metrics.Dice(threshold=0.5, activation='sigmoid'),
    utils.metrics.Fscore(threshold=None, activation='sigmoid')
]
optimizer = torch.optim.Adam([
    dict(params=model.parameters(), lr=lr),
])

train_epoch = utils.train.TrainEpoch(
    model,
    loss=loss,
    metrics=metrics,
    optimizer=optimizer,
    device=device,
    verbose=True,
)

In [98]:
max_score = 0
os.makedirs(dir_checkpoint, exist_ok=True)

In [74]:
import sys
import torch
from tqdm import tqdm as tqdm
from utils.meter import AverageValueMeter

logs = {}
loss_meter = AverageValueMeter()
metrics_meters = {
    metric.__name__: AverageValueMeter()
    for metric in metrics
}

stage_name='train'
verbose=True

with tqdm(train_loader,
          desc=stage_name,
          file=sys.stdout,
          disable=not (verbose)) as iterator:
    for x1, y1 in iterator:
        x1, y1 = x1.to(device), y1.to(device)
        print(x1.shape)
        print(type(x1))
        print(y1.shape)
        break

train:   0%|          | 0/18 [00:00<?, ?it/s][  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 2

In [67]:
import torchvision.transforms as T
transform = T.ToPILImage()
img = transform(x1.squeeze(dim=0))
img.show()

In [75]:
# trian batch_update
optimizer.zero_grad()

In [ ]:
prediction = model.forward(x1)

In [80]:
loss = loss(prediction, y1)

In [82]:
loss.backward()

In [83]:
optimizer.step()

In [84]:
y_pred = prediction

In [85]:
loss_value = loss.cpu().detach().numpy()
loss_value

array(0.88039684, dtype=float32)

In [87]:
loss_meter.add(loss_value)
loss_meter

### val_epoch

In [6]:
model=net
loss=utils.losses.NoiseRobustDiceLoss(eps=1e-7, activation='sigmoid')
metrics=[
    utils.metrics.Dice(threshold=0.5, activation='sigmoid'),
    utils.metrics.Fscore(threshold=None, activation='sigmoid')
]
stage_name='valid'
device='cpu'
verbose='True'
valid_epoch = utils.train.ValidEpoch(
    model,
    loss=loss,
    metrics=metrics,
    device=device,
    verbose=True,
)

def _format_logs(logs):
    str_logs = ['{} - {:.4}'.format(k, v) for k, v in logs.items()]
    s = ', '.join(str_logs)
    return s


In [ ]:
with tqdm(val_loader,
          desc=stage_name,
          file=sys.stdout,
          disable=not (verbose)) as iterator:
    for x, y in iterator:
        x, y = x.to(device), y.to(device)
        break

In [52]:
transform = T.ToPILImage()
img = transform(x.squeeze(dim=0))
img.show()



In [58]:
prediction.shape

torch.Size([1, 1, 400, 400])

In [60]:
loss = utils.losses.NoiseRobustDiceLoss(eps=1e-7, activation='sigmoid')
loss.to(device)
loss = loss(prediction, y2)

